# Initialization

In [1]:
using CSV, DataFrames, JuMP, HiGHS, LinearAlgebra, DataStructures, Statistics
baltic_demand = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/Demand_Baltic.csv", delim='\t'))
ports = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/ports.csv", delim='\t'))
distances = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/dist_dense.csv", delim='\t'))
baltic_fleet = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_Baltic.csv", delim='\t'))
fleet_data = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_data.csv", delim='\t'))
;

In [2]:
commodities = []
for i in 1:size(baltic_demand, 1)
    push!(commodities, string(baltic_demand.Origin[i], "s", "|", baltic_demand.Destination[i], "t"))
end

### Services:

The following array contains manual generated services to which the column generation will be applied.

In [3]:
# services = [["Feeder_800" "DEBRV" "NOKRS" "DEBRV"],
#             ["Feeder_800" "RUKGD" "FIKTK" "RUKGD"],
#             ["Feeder_450" "DEBRV" "NOSVG" "DEBRV"],
#             ["Feeder_450" "NOAES" "NOBGO" "NOAES"],
#             ["Feeder_450" "NOKRS" "SEGOT" "DKAAR" "NOKRS"],
#             ["Feeder_800" "DEBRV" "SEGOT" "DKAAR" "DEBRV"],
#             ["Feeder_450" "DEBRV" "NOSVG" "NOBGO" "NOAES" "DEBRV"],
#             ["Feeder_450" "SEGOT" "DKAAR" "PLGDY" "RUKGD" "SEGOT"],
#             ["Feeder_450" "DEBRV" "NOAES" "DEBRV" "RULED" "DEBRV"],
#             ["Feeder_450" "DEBRV" "RUKGD" "PLGDY" "DKAAR" "SEGOT" "DEBRV"],
#             ["Feeder_800" "PLGDY" "RUKGD" "RULED" "FIKTK" "FIRAU" "PLGDY"],
#             ["Feeder_450" "PLGDY" "RUKGD" "FIKTK" "RULED" "FIRAU" "PLGDY"],
#             ["Feeder_450" "NOKRS" "SEGOT" "DKAAR" "PLGDY" "RUKGD" "PLGDY" "DKAAR" "SEGOT" "NOKRS"],
#             ["Feeder_800" "NOAES" "NOBGO" "NOSVG" "NOKRS" "SEGOT" "DKAAR" "SEGOT" "NOKRS" "NOSVG" "NOBGO" "NOAES"]]

# services = [[33.0 2 "Feeder_800" "DEBRV" "NOKRS" "DEBRV"],
#                 [33.0 2 "Feeder_800" "NOKRS" "NOSVG" "NOBGO" "NOAES" "NOKRS"],
#                 [33.0 2 "Feeder_450" "NOKRS" "SEGOT" "DKAAR" "NOKRS"],
#                 [33.0 2 "Feeder_800" "DKAAR" "PLGDY" "RUKGD" "DKAAR"],
#                 [33.0 2 "Feeder_450" "SEGOT" "FIRAU" "FIKTK" "RULED" "SEGOT"]]

# services = [[33.0 2 "Feeder" "DEBRV" "DKAAR" "SEGOT" "FIRAU" "DEBRV"],
#             [33.0 2 "Feeder" "NOKRS" "DKAAR" "SEGOT" "NOBGO" "NOKRS"]]

services = [
    [110000 1 "Feeder_450" "DEBRV" "NOSVG" "NOBGO" "NOAES" "DEBRV"],
    [100000 1 "Feeder_450" "DEBRV" "RUKGD" "FIKTK" "FIRAU" "PLGDY" "DEBRV"],
    [90000 1 "Feeder_450" "DEBRV" "DKAAR" "SEGOT" "NOKRS" "DEBRV"],
    [970000 1 "Feeder_450" "DEBRV" "SEGOT" "RULED" "DEBRV"]
]


# Initialize an empty Set to store unique ports
service_ports = []
# Iterate through the subarrays and add the unique ports to the set
for service in services
    for port in service[4:end]
        push!(service_ports, port)
    end
end
service_ports = unique(service_ports);


In [4]:
# List of ports that have intersecting service
intersect_ports = []
for i in 1:length(services)
    for j in i+1:length(services)
        push!(intersect_ports, intersect([[row[4:end]...] for row in services][i], [[row[4:end]...] for row in services][j]))
    end
end
intersect_ports = collect(Set(i for j in intersect_ports for i in j)); # Collecting unique transshipment nodes

# Adding transshipment nodes where services intersect
services_w_transshipment = []
transshipment_nodes = []
i = 1
for service in services
    updated_service = []
    for port in service
        if port in intersect_ports
            push!(updated_service, port * string(i)) # vi behøves i realiteten ikke at update vores serivces
            push!(transshipment_nodes, port * string(i))
        else
            push!(updated_service, port)
        end
    end
    push!(services_w_transshipment, updated_service)
    i += 1
end
transshipment_nodes = unique(transshipment_nodes);

In [5]:
# Source and terminal adjacency list
s_t_adj = []
for port in service_ports
    for node in transshipment_nodes
        if port == node[1:5]
            push!(s_t_adj, [string(port, "s"), node, parse.(Float64, ports[ports.UNLocode .== port, 9][1])  + ports[ports.UNLocode .== port, 12][1]])
            push!(s_t_adj, [node, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1]) + ports[ports.UNLocode .== port, 12][1]])
        end
    end
    if port ∉ intersect_ports
        push!(s_t_adj, [string(port, "s"), port, parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
        push!(s_t_adj, [port, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
    end
end

# Transshipment nodes adjacency list
transshipment_adj = []
for node in intersect_ports, i in transshipment_nodes, j in transshipment_nodes
    if j!=i && contains(i, node) && contains(j, node)
        push!(transshipment_adj, [i, j, parse.(Float64, ports[ports.UNLocode .== node, 10][1])]) # Skal vi også have move omkostninger med, da vi skal load og disch?
    end
end

# Service ports adjacency list
ports_adj = []
test = []
for service in 1:length(services_w_transshipment)
    for port in 5:length(services_w_transshipment[service]) # Hvad gør vi med port call omkostninger?
        push!(ports_adj, [services_w_transshipment[service][port-1], services_w_transshipment[service][port], 0.0]) # ports[ports.UNLocode .== services_w_transshipment[service][port][1:5], 12][1]]
    end
end

# Penalty edges adjacency list
penalty_adj = []
for k in 1:size(baltic_demand,1)
    push!(penalty_adj, [string(baltic_demand.Origin[k], "s"), string(baltic_demand.Destination[k], "t"), 1000])
end

dummy_adj = []
for port in 1:length(service_ports)
    push!(dummy_adj, [string(service_ports[port], "t"), nothing, 0.0])
end


edges = vcat(s_t_adj, transshipment_adj, ports_adj, penalty_adj, dummy_adj);

In [7]:
function get_G(edges)
    # Create an empty adjacency list
    graph = Dict{String, Vector{Tuple{Any, Float64}}}()  # Use Tuple to store (target, VC)
    #graph = Dict{String, Vector{Tuple{String, Float64}}}()  # Use Tuple to store (target, VC)

    # Populate the adjacency list
    for edge in edges
        source, target, weight = edge
        if haskey(graph, source)
            push!(graph[source], (target, weight))
        else
            graph[source] = [(target, weight)]
        end
    end

    return graph
end

get_G (generic function with 1 method)

In [8]:
function dijkstra(G, source, destination) # HUSK DUAL VARIABLES
    dist = Dict{Any, Float64}()
    prev = Dict{Any, Any}()
    Q = Set(keys(G))

    for v in Q
        dist[v] = Inf
        prev[v] = nothing
    end

    dist[source] = 0.0

    while !isempty(Q)
        min_value = Inf
        min_key = string()
        for i in Q
            if dist[i] <= min_value
                min_value = dist[i]
                min_key = i
            end
        end
        u = min_key

        if dist[u] == Inf
            break
        end
        if u == destination
            break
        end
        
        delete!(Q, u)

        for v in G[u]
            if haskey(G, v[1])
                alt = dist[u] + v[2]
                if alt < dist[v[1]]
                    dist[v[1]] = alt
                    prev[v[1]] = u
                end
            else
                break
            end
        end
    end

    S = []
    u = destination
    push!(S, u)

    while prev[u] != nothing
        push!(S, u)
        u = prev[u]
    end
    push!(S, source)
    
    return reverse(S[2:end]), dist[destination]
end


dijkstra (generic function with 1 method)

## Flowproblem

### Initialization

In [51]:
paths = []
costs = []
;

In [52]:
# Decision variables
i_j = []
for edge in edges
    if edge ∈ (penalty_adj ∪ dummy_adj)
        break
    end
    push!(i_j, string(edge[1], "|", edge[2]))
end

# Initializing parameters
dual_var = zeros(length(edges))

# Updated edges w.r.t. dual costs
edges_dual = edges
for i in 1:length(edges_dual)
    edges_dual[i][3] = edges_dual[i][3] + dual_var[i]
end

for i in penalty_adj
    push!(paths, i[1:2])
    push!(costs, i[3])
end

# Paths and costs from shortest path
for i in 1:size(baltic_demand, 1)
    path, cost = dijkstra(get_G(edges_dual), string(baltic_demand.Origin[i],"s"), string(baltic_demand.Destination[i],"t"))
    push!(paths, path)
    push!(costs, cost)
end


# Initializing parameters
n_k = size(baltic_demand.Origin, 1) # Number of commodities
K = 1:n_k

n_s = size(services_w_transshipment, 1) # Number of services
S = 1:n_s

n_ff = size(penalty_adj, 1) # Number of penalty edges
ff = 1:n_ff
index_ff = 1:n_k # Index of ff paths - Could be hard coded

n_p = size(paths, 1) # Number of paths
P = 1:n_p
index_p = n_k+1:n_p  # Index of non-ff paths - Could be hardcoded

n_ij = size(i_j, 1) # Number of edges
ij = 1:n_ij # Range of edges
;

In [53]:
# Initial paths including shortest path for all commodities and activated penalty edges
aij = Matrix{Float64}(undef, n_ij, 0) # 1 if path p for commodity k uses edge (i,j), else 0
uij = Matrix{Float64}(undef, n_ij, 0) # vessel capacity if service s uses edge (i,j), else 0
PK = Matrix{Float64}(undef, n_k, 0) # Type of commodity for each path

for i in index_p
    ak = zeros(Float64, n_ij)
    for j in 2:length(paths[i])
        ak[findall(x -> x == string(paths[i][j-1], "|", paths[i][j]), i_j)[1]] = 1
    end
    aij = [aij ak]
end

for i in 1:length(services_w_transshipment)
    us = zeros(Float64, n_ij)
    for j in 5:length(services_w_transshipment[i])
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j-1][1:5], "t"), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
        us[findall(x -> x == string(services_w_transshipment[i][j-1][1:5], "s", "|", services_w_transshipment[i][j-1]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
    end
    uij = [uij us]
end

for i in 1:length(paths)
    pk = zeros(Float64, n_k)
    pk[findall(x -> x == string(paths[i][1], "|", paths[i][end]), commodities)[1]] = 1
    PK = [PK pk]
end

### Actual problem

In [54]:
flowModel = Model(HiGHS.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, 0 <= y[S] <= 1)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum(costs[j]*x[j] for j in P))
@constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j+n_ff] for j in K) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
# Skal vi have constraints på s -> t edges/paths
@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
print(flowModel)
#optimize!(flowModel)
#solution_summary(flowModel)




Min 110000 y[1] + 100000 y[2] + 90000 y[3] + 970000 y[4] + 1000 x[1] + 1000 x[2] + 1000 x[3] + 1000 x[4] + 1000 x[5] + 1000 x[6] + 1000 x[7] + 1000 x[8] + 1000 x[9] + 1000 x[10] + 1000 x[11] + 1000 x[12] + 1000 x[13] + 1000 x[14] + 1000 x[15] + 1000 x[16] + 1000 x[17] + 1000 x[18] + 1000 x[19] + 1000 x[20] + 1000 x[21] + 1000 x[22] + 409 x[23] + 642 x[24] + 528 x[25] + 446 x[26] + 897 x[27] + 297 x[28] + 473 x[29] + 578 x[30] + 446 x[31] + 409 x[32] + 354 x[33] + 578 x[34] + 350 x[35] + 897 x[36] + 297 x[37] + 473 x[38] + 528 x[39] + 350 x[40] + 642 x[41] + 483 x[42] + 354 x[43] + 483 x[44]
Subject to
 Y_k[1] : x[1] + x[23] = 77
 Y_k[2] : x[2] + x[24] = 456
 Y_k[3] : x[3] + x[25] = 65
 Y_k[4] : x[4] + x[26] = 7
 Y_k[5] : x[5] + x[27] = 10
 Y_k[6] : x[6] + x[28] = 98
 Y_k[7] : x[7] + x[29] = 660
 Y_k[8] : x[8] + x[30] = 17
 Y_k[9] : x[9] + x[31] = 268
 Y_k[10] : x[10] + x[32] = 18
 Y_k[11] : x[11] + x[33] = 16
 Y_k[12] : x[12] + x[34] = 37
 Y_k[13] : x[13] + x[35] = 187
 Y_k[14] : x[14]

In [13]:
duals_constraints_Y_ij = [dual(flowModel[:Y_ij][i]) for i in 1:n_ij]
duals_constraints_Y_k = [dual(flowModel[:Y_k][i]) for i in 1:n_k]

primal_values_x = value.(x[p])
primal_values_y = value.(y[s])

[-141.44444444444446, -103.0, -554.0, -591.0, -358.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -527.0, 0.0, 0.0, -517.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -527.0, 0.0]
[1000.0, 1000.0, 669.4444444444445, 1000.0, 1000.0, 851.0, 1000.0, 719.4444444444445, 1000.0, 963.0, 354.0, 681.0, 904.0, 1000.0, 888.0, 1000.0, 631.0, 941.0, 642.0, 1000.0, 712.0, 1000.0]


Berfore the master problem is formulated, the $c_{y}$ constants are calculated which correspond to the fixed port costs (more might be added later).

Moreover, the sum of all paths must be equal to $d_{k}$, in order to fulfill the demand. However later on, a new expensive penalty edge is added which penalizes all the containers that are not delivered to the commodity port.

As the decision variables, $x_{i,j}$ represent the edges in the liner shipping network (eventually paths), the variables are initialized by pairing the neigbor ports in the services above. Thereby, the master problem is reduced to only respect the strict subset of all feasible patterns i.e. patterns using the generated services.